# Billboard Hot 100 Turns 60: Top 600 Songs Of All Time

In honor of the 60th Anniversary of the Billboard Hot 100 Singles chart, on August 4, 2018, 
they created this [special chart of the 600 biggest songs of all time.](https://open.spotify.com/playlist/0X9hkrRqCCP69Ze1MheAda)

We'll use Spotipy library API for scraping data from Spotify. Besides extracting the 600 songs from the Billboard, we'll also pull interesting song data such as danceability, loudness, energy of each song.

In [95]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from configparser import ConfigParser
import pandas as pd
from pprint import pprint
import datetime
from datetime import timedelta

## Data Extraction

In [96]:
config = ConfigParser()
config.read('notebook.cfg')
client_id = config['spotify_api']['client_id']
client_secret = config['spotify_api']['client_secret']

#Authenticating with Spotipy
client_credentials_manager = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

### 1. Playlist Items
Since playlist_items() function is limited to 100 tracks, we'll create a custom method to pull all tracks in [the Top 600 Songs list](https://open.spotify.com/playlist/0X9hkrRqCCP69Ze1MheAda)

In [97]:
# Playlist_tracks
def get_playlist_tracks(playlist_id):
    
    pl_id = 'spotify:playlist:'+ playlist_id
    offset = 0
    print(pl_id)
    while True:
        response = sp.playlist_items(pl_id,
                                 offset=offset,
                                 fields='items(track(id,name,artists(name),album(name, album_type, release_date),popularity,duration_ms))'
                                    )
    
        if len(response['items']) == 0:
            break
        
        for item in response['items']:
            yield(item["track"])

        offset = offset + len(response['items'])

In [98]:
playlist_link = 'https://open.spotify.com/playlist/0X9hkrRqCCP69Ze1MheAda'
playlist_id = playlist_link.split("/")[-1].split("?")[0]

In [99]:
track_list = [] 
for track in get_playlist_tracks(playlist_id):
    
    track_id = track['id']
    track_name = track['name']
     
    artist_name = track['artists'][0]['name']
   
    album = track['album']['name']
    album_type = track['album']['album_type']
    release_date = track['album']['release_date']

    track_pop = track['popularity']
    track_duration = track['duration_ms']
    
    newlist =  [track_id, track_name, track_duration, track_pop, artist_name, album, album_type, release_date ]
    
    track_list.append(newlist)

spotify:playlist:0X9hkrRqCCP69Ze1MheAda


In [100]:
#track_list

In [101]:
playlist_df = pd.DataFrame(track_list)
playlist_df.columns = ['track_id', 'track_name', 'duration', 'track_pop', 'artist', 'album', 'album_type', 'release_date']
#playlist_df.info()

### Data transformation

In [102]:
#Convert 'duration' from milliseconds to seconds.
playlist_df['duration_sec'] = pd.to_timedelta(playlist_df.duration, unit='ms')
playlist_df.duration_sec = playlist_df.duration_sec.dt.total_seconds().astype(int)
#playlist_df.head()

Another option is to convert 'duration' column to a easy-to-read format using datetime module.

In [103]:
playlist_df['duration'] = pd.to_datetime(playlist_df['duration'],
             unit='ms').dt.strftime('%M:%S:%f').str[:5]

In [104]:
columns = ['track_id', 'track_name','track_pop', 'duration','duration_sec', 'artist', 'album', 'album_type', 'release_date']
playlist_df = playlist_df[columns]

In [105]:
playlist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   track_id      600 non-null    object
 1   track_name    600 non-null    object
 2   track_pop     600 non-null    int64 
 3   duration      600 non-null    object
 4   duration_sec  600 non-null    int64 
 5   artist        600 non-null    object
 6   album         600 non-null    object
 7   album_type    600 non-null    object
 8   release_date  600 non-null    object
dtypes: int64(2), object(7)
memory usage: 42.3+ KB


### 2.Track Audio Features
In order to pull the corresponding audio features data for [the Top 600 Songs list](https://open.spotify.com/playlist/0X9hkrRqCCP69Ze1MheAda), we'll use the audio_features() function.

In [106]:
track_ids = []

for track in track_list:
    track_id = track[0]
    track_ids.append(track_id)

len(track_ids)

600

In [107]:
audiofeatures_list = [] 

for track_id in track_ids :
    features = sp.audio_features(track_id)[0]
    danc = features['danceability']
    enrg = features['energy']
    key = features['key']
    loud = features['loudness']
    mode = features['mode']
    spch = features['speechiness']
    acou = features['acousticness']
    inst = features['instrumentalness']
    live = features['liveness']
    valn = features['valence']
    temp = features['tempo']
    
    flist = [track_id, danc, enrg, key, loud, mode, spch, acou, inst, live, valn, temp]
    audiofeatures_list.append(flist)

In [108]:
features_df = pd.DataFrame(audiofeatures_list)
features_df.columns = ['track_id', 'danc', 'enrg', 'key', 'loud', 'mode', 'spch', 'acou', 'inst', 'live', 'valn', 'temp']

In [109]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   track_id  600 non-null    object 
 1   danc      600 non-null    float64
 2   enrg      600 non-null    float64
 3   key       600 non-null    int64  
 4   loud      600 non-null    float64
 5   mode      600 non-null    int64  
 6   spch      600 non-null    float64
 7   acou      600 non-null    float64
 8   inst      600 non-null    float64
 9   live      600 non-null    float64
 10  valn      600 non-null    float64
 11  temp      600 non-null    float64
dtypes: float64(9), int64(2), object(1)
memory usage: 56.4+ KB


In [110]:
top600_list= pd.merge(playlist_df, features_df, how="inner")

In [111]:
top600_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 0 to 599
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   track_id      600 non-null    object 
 1   track_name    600 non-null    object 
 2   track_pop     600 non-null    int64  
 3   duration      600 non-null    object 
 4   duration_sec  600 non-null    int64  
 5   artist        600 non-null    object 
 6   album         600 non-null    object 
 7   album_type    600 non-null    object 
 8   release_date  600 non-null    object 
 9   danc          600 non-null    float64
 10  enrg          600 non-null    float64
 11  key           600 non-null    int64  
 12  loud          600 non-null    float64
 13  mode          600 non-null    int64  
 14  spch          600 non-null    float64
 15  acou          600 non-null    float64
 16  inst          600 non-null    float64
 17  live          600 non-null    float64
 18  valn          600 non-null    